In [1]:
import sqlite3
import pandas as pd
from IPython.display import display

# Conecta ao banco na raiz do projeto
conn = sqlite3.connect("../unidades.db")

# Consulta colunas necessárias, incluindo latitude e longitude para filtragem
query = """
SELECT rede, nome, endereco, latitude, longitude
FROM unidades
"""

df_unidades = pd.read_sql_query(query, conn)
conn.close()

# Filtra apenas registros sem latitude ou longitude
df_sem_coords = df_unidades[(df_unidades['latitude'].isnull()) | (df_unidades['latitude'] == '') |
                            (df_unidades['longitude'].isnull()) | (df_unidades['longitude'] == '')]

print(f"Total de registros sem coordenadas: {len(df_sem_coords)}")

# Exibe apenas as colunas que você quer visualizar
display(df_sem_coords[['rede', 'nome', 'endereco', 'latitude','longitude']])


Total de registros sem coordenadas: 0


,rede,nome,endereco,latitude,longitude


In [1]:
import sqlite3
import pandas as pd

# Conecta ao banco na raiz do projeto
conn = sqlite3.connect("../unidades.db")

# Consulta todas as colunas relevantes
query = """
SELECT id, rede, nome, endereco, latitude, longitude,
       bairro_cdn, cidade_cdn, estado_cdn, pais_cdn
FROM unidades
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Função para contar valores "faltantes" (NULL ou string vazia)
def contar_faltantes(col):
    return df[col].isnull().sum() + (df[col] == '').sum()

# Lista de colunas que queremos resumir
colunas_coords = ['latitude', 'longitude']
colunas_cdns = ['bairro_cdn', 'cidade_cdn', 'estado_cdn', 'pais_cdn']

print("📊 Resumo de valores faltantes por coluna:\n")

# Resumo para coordenadas
print("➤ Coordenadas:")
for col in colunas_coords:
    faltantes = contar_faltantes(col)
    total = len(df)
    print(f"  {col}: {faltantes} / {total} ({faltantes/total:.2%}) faltantes")

print("\n➤ CDN (bairro, cidade, estado, país):")
for col in colunas_cdns:
    faltantes = contar_faltantes(col)
    total = len(df)
    print(f"  {col}: {faltantes} / {total} ({faltantes/total:.2%}) faltantes")

# Estatística combinada: quantos registros têm algum coordenada ou algum CDN faltando
faltando_coords = df[colunas_coords].isnull() | (df[colunas_coords] == '')
faltando_cdns = df[colunas_cdns].isnull() | (df[colunas_cdns] == '')

total_faltando_coords = faltando_coords.any(axis=1).sum()
total_faltando_cdns = faltando_cdns.any(axis=1).sum()
total_faltando_qualquer = (faltando_coords.any(axis=1) | faltando_cdns.any(axis=1)).sum()

print("\n📌 Resumo combinado:")
print(f"  Registros com algum valor de coordenada faltando: {total_faltando_coords} / {len(df)}")
print(f"  Registros com algum valor de CDN faltando: {total_faltando_cdns} / {len(df)}")
print(f"  Registros com algum valor faltando (coords ou CDN): {total_faltando_qualquer} / {len(df)}")


📊 Resumo de valores faltantes por coluna:

➤ Coordenadas:
  latitude: 0 / 2179 (0.00%) faltantes
  longitude: 0 / 2179 (0.00%) faltantes

➤ CDN (bairro, cidade, estado, país):
  bairro_cdn: 55 / 2179 (2.52%) faltantes
  cidade_cdn: 9 / 2179 (0.41%) faltantes
  estado_cdn: 9 / 2179 (0.41%) faltantes
  pais_cdn: 9 / 2179 (0.41%) faltantes

📌 Resumo combinado:
  Registros com algum valor de coordenada faltando: 0 / 2179
  Registros com algum valor de CDN faltando: 55 / 2179
  Registros com algum valor faltando (coords ou CDN): 55 / 2179
